# Overview

This notebook demonstrates how a YAML file with a new timbre can be created based on a WAV file with recording of a real instrument. Such approach is sometimes called resynthesis.

Although synthesized with this procedure sounds are not similar enough to original sounds, created presets may be used for further manual editing. 

# Setup

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import numpy as np
import wavio
from scipy.signal import spectrogram

In [2]:
note = 'C4'
fundamental_frequency = 261.6
duration_in_seconds = 2
frame_rate = 44100

In [3]:
duration_in_frames = int(round(frame_rate * duration_in_seconds))

In [4]:
target = wavio.read(f'nylon_guitar_samples/{note}.wav').data
target = np.transpose(target)
target = target / np.max(np.abs(target))  # Scale values to be not greater than 1.
target = target[0, :duration_in_frames]
padding = np.array([0 for _ in range(duration_in_frames - target.shape[0])])
target = np.hstack((target, padding))
target.shape

(88200,)

# Spectrogram

In [5]:
n_first_frequencies = 200
n_first_frames = 50
frequencies, time_frames, spc = spectrogram(target, frame_rate, nperseg=2205, nfft=4410)
spc = np.sqrt(spc)  # Initially, there is power which is proportional to square of amplitude.

In [ ]:
fig = plt.figure(figsize=(16, 32))
ax = fig.add_subplot(111)
sns.heatmap(
    spc[:n_first_frequencies, :n_first_frames],
    yticklabels=frequencies[:n_first_frequencies],
    ax=ax, cmap='coolwarm'
)

# Config Generation

In [6]:
threshold = 0.00005
envelopes = {}
for i, frequency in enumerate(frequencies.tolist()[:n_first_frequencies]):
    partial = frequency / fundamental_frequency
    harmonic_partial = round(frequency / fundamental_frequency)
    if partial < 1:
        continue
    # Some values may be located to slightly wrong frequencies. In particular, 
    # this is due to assumption that sound pressure is periodic on every time segment of spectrogram.
    # So combine back harmonic partials.
    if abs(frequency - harmonic_partial * fundamental_frequency) / fundamental_frequency < 0.05:
        envelope = envelopes.get(harmonic_partial, [])
        if envelope:
            envelope = [x + y for x, y in zip(envelope, spc[i, :].tolist())]
        else:
            envelope = spc[i, :].tolist()
        envelopes[harmonic_partial] = envelope
    # Inharmonic partials are important for attack.
    else:
        envelope = spc[i, :].tolist()
        if max(envelope) > threshold:
            envelopes[partial] = envelope

In [7]:
volumes = {k: max(v) for k, v in envelopes.items()}
envelopes = {k: [x / volumes[k] for x in v] for k, v in envelopes.items()}
fundamental_volume = volumes.pop(1)
volume_ratios = {k: v / fundamental_volume for k, v in volumes.items()}

In [8]:
attack_in_elements = 2
max_attack_duration = 0.1

In [9]:
config = '''
---
- name: generated_timbre
  fundamental_waveform: sine
  fundamental_volume_envelope:
    name: user_defined
    parts:
      - values:
          - 0
          - {}
        max_duration: {}
      - values:
          - {}
          - 0
        max_duration: null
  fundamental_effects:
    - name: vibrato
      frequency: 4
      width: 0.05
    - name: tremolo
      frequency: 3
      amplitude: 0.05
  overtones_specs:
'''.format(
    '\n          - '.join([str(float(x)) for x in envelopes[1][:attack_in_elements]]),
    max_attack_duration,
    '\n          - '.join([str(float(x)) for x in envelopes[1][attack_in_elements-1:]])
)
config = config[1:-1]  # Strip extra '\n'.
for k, v in envelopes.items():
    if k == 1:
        continue
    continuation = '''
    - waveform: sine
      frequency_ratio: {}
      volume_ratio: {}
      volume_envelope:
        name: user_defined
        parts:
          - values:
              - 0
              - {}
            max_duration: {}
          - values:
              - {}
              - 0
            max_duration: null
      effects:
      - name: vibrato
        frequency: 4
        width: 0.05
      - name: tremolo
        frequency: 3
        amplitude: 0.05
    '''.format(
        k,
        volume_ratios[k],
        '\n              - '.join([str(float(x)) for x in envelopes[k][:attack_in_elements]]),
        max_attack_duration,
        '\n              - '.join([str(float(x)) for x in envelopes[k][attack_in_elements-1:]])
    )
    continuation = continuation[:-5]  # Strip spaces.
    config += continuation

In [10]:
with open('generated_timbre.yml', 'w') as out_file:
    for line in config:
        out_file.write(line)